In [45]:
import os

In [46]:
%pwd

'f:\\DS MLOps\\dsproject'

In [47]:
os.chdir("f:/DS MLOps/dsproject")

In [48]:
%pwd

'f:\\DS MLOps\\dsproject'

In [49]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [50]:
from src.dsproject.constants import * #CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from src.dsproject.utils.common import read_yaml, create_directories

In [51]:
class DataConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config['artifacts_root']])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
            config= self.config.data_ingestion
            create_directories([config.root_dir])
            data_ingestion_config= DataIngestionConfig(
                root_dir= config.root_dir,
                source_url= config.source_URL,
                local_data_file= config.local_data_file,
                unzip_dir= config.unzip_dir
            )
            return data_ingestion_config
        
    
        

In [52]:
import os
import urllib.request as request
from src.dsproject import logger
import zipfile

In [55]:
# Data Ingestion components
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    # Download the zip file
    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_url, 
                filename=self.config.local_data_file
            )
            logger.info(f"File: {filename} downloaded with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {os.path.getsize(self.config.local_data_file)}")
    # Unzip the downloaded file
    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [56]:
try:
    config=DataConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
    logger.info("Data Ingestion completed successfully.")
except Exception as e:
    raise e

[2025-11-09 16:20:45,606: INFO: common: YAML file: config\config.yaml loaded successfully]
[2025-11-09 16:20:45,608: INFO: common: YAML file: params.yaml loaded successfully]
[2025-11-09 16:20:45,610: INFO: common: YAML file: schema.yaml loaded successfully]
[2025-11-09 16:20:45,611: INFO: common: Directory created at: artifacts]
[2025-11-09 16:20:45,612: INFO: common: Directory created at: artifacts/data_ingestion]
[2025-11-09 16:20:45,614: INFO: 938239821: File already exists of size: 23329]
[2025-11-09 16:20:45,766: INFO: 2064736844: Data Ingestion completed successfully.]
